In [13]:
from astropy.constants import G, c, M_sun, R_sun
from math import pi
from astropy import units as u
from math import pow
import numpy as np

In [61]:
m1 = 10.8 * u.Msun
m2 = 31.79 * u.Msun
period = 3.909 * u.day
omega = (2 * pi / period)
r2 = 17.0 * u.Rsun

vwind_inf = 1400 * u.km / u.s
inclin = 36.38 * u.deg
r2_rot = ((130) * u.km / u.s) / np.sin(inclin)

period = period.to(u.yr)
a = (((m1 + m2) * period**2)**(1/3)).value * u.au
print(f'Separation: {a:8.4}')
print(f'Separation (light seconds): {a.to(u.m)/c:8.4}')

# BH orbital velocity:
orb_vel = np.sqrt(G * (m1 + m2).to(u.kg) / a.to(u.m))
orb_vel = orb_vel.to(u.km / u.s)
print(f"Orbital velocity: {orb_vel:6.5}")
print()

# Non-spinning black hole r_G
rG = (2 * m1.to(u.kg) * G) / (c**2)
print(f'Schwarzschild Radius: {rG:10.7}')
print(f'Schwarzschild Radius: {rG.to(u.earthRad):8.3}')

Separation:   0.1696 AU
Separation (light seconds):    84.63 s
Orbital velocity: 471.99 km / s

Schwarzschild Radius:    31895.1 m
Schwarzschild Radius:    0.005 earthRad


In [15]:

def wind_velocity(r, beta = 1.0, v_inf = 1000*u.km / u.s):
    '''
    From Negueruela eqn 1. Computes the wind velocity for a line driven
    wind as a function of distance from the star.
    
    Parameters
    ----------
    r: float
        Distane, scaled by the radius of the OB star (i.e. r = r_d / R_star)

    '''
    
    return v_inf * (1 - r**-1)**(beta)



In [17]:
# Tangential rotational velocity at BH:
v_tan = orb_vel -  (r2 / a.to(u.Rsun))* r2_rot

wind_speed = wind_velocity(a.to(u.m) / r2.to(u.m), v_inf =vwind_inf )
print(f'Wind speec at BH: {wind_speed}')
v_rel = np.sqrt(wind_speed**2 + v_tan**2)
print(f'Relative local speed of gas to BH: {v_rel:8.4}')

r_acc = (2 * G * m1.to(u.kg) / (v_rel**2)).to(u.Rsun)
print(f'Accretion radius: {r_acc:9.4}')


Wind speec at BH: 747.389528819054 km / s
Relative local speed of gas to BH:    833.9 km / s
Accretion radius:     5.926 solRad


Okay, now play with critical angular momentum from Beloborodov 2001

 https://ui.adsabs.harvard.edu/abs/2001MNRAS.323..167B/abstract



In [41]:
lstar = 0.75 * rG * c
eta = 1

l_ave = (eta * 0.25 * omega * (r_acc**2)).to(u.m**2 / u.s)

print(l_ave / lstar)




11.021531505694373


Now look at the mass accretion rate:

In [42]:
# M = M =4.52 ́10-7M yr from Vink et al 2001:
m2_massloss = 4.52e-7 * u.Msun / u.yr
density = (m2_massloss / (4 * np.pi * a**2 * wind_speed)).to(u.g/u.cm**3)
print(f'Density of wind at BH: {density:8.4}')

# Bondi-Hoyle 1944:

m1_mass_rate = (np.pi * r_acc**2 * density * v_rel).to(u.Msun / u.yr)
print(f'Accretion rate: {m1_mass_rate:9.4}')

Density of wind at BH: 4.711e-15 g / cm3
Accretion rate: 3.329e-09 solMass / yr


In [113]:
# Figure out what the outer diameter of the accretion disk is supposed to be
r_d = 0.25 * (v_rel * a.to(u.km))**2 * (r_acc.to(u.km) / a.to(u.km))**4 / (G * m1.to(u.kg))
my_rd = ( (r_d.to(u.m) / rG))

# Outer diameter at 1700 rG. 

# From Shapiro 1976
v_esc = (((2 * G * m1.to(u.kg) / r2.to(u.m))**0.5)).to(u.km / u.s)
ratio_to_isco = 160 * ( (v_esc / wind_speed)**8)
ratio_to_isco *=  ( ( 1 + (orb_vel/wind_speed)**2) **(-4))
ratio_to_isco *= (m2 / (10*u.Msun))**2
ratio_to_isco *= (m1 / (30*u.Msun))**-4
ratio_to_isco *= (period.to(u.d) / (5.6*u.d))**(-4/3)
ratio_to_isco *= (m2_massloss / (1e-6 * u.Msun / u.yr))

In [115]:
print(my_rd / ratio_to_isco)

2.6222503634184418


In [135]:
# Free fall time at R_acc (Illarionov 1974)
t = (r_acc.to(u.m)**(3/2)) / (2 * G * m1.to(u.kg))**0.5
# Cooling time due to bremsstrahlung
# Accretion shock temperature:
shock_temp = 4e7 * (wind_speed.to(u.cm / u.s).value * 1e-8)**2 * (r_acc / r_acc) * u.K
t_bremss = 3e11 * (shock_temp**0.5) / density

In [136]:
t_bremss

<Quantity 3.01016488e+29 cm3 K(1/2) / g>